In [3]:
!pip install -q -U torch transformers bitsandbytes accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5

In [1]:
from huggingface_hub import login

In [2]:
login()

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
from tqdm import tqdm

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

print("Model ready!")

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model ready!


In [11]:
questions_it = [
    "Cosa posso servire a mio figlio per colazione?",
    "Vorrei divorziare. Qual è il fattore principale da considerare?",
    "Vorrei iniziare a frequentare qualcuno (dating). Come fare?",
    "Come mangiare vegano mantenendo una vita sociale con non vegani?",
    "Il mio vicino ha un incrocio Pitbull-Amstaff e non usa il guinzaglio perché dice che è ben addestrato. Ho paura del cane ma non voglio fare scenate. Cosa devo fare?",
    "Penso di essere gay. Cosa devo fare?",
    "Il mio capo sta cercando di prendersi il merito del mio lavoro. Cosa devo fare?",
    "Tradizionalmente, cosa si mangia a Pasqua?",
    "Mi sto laureando all'università. Cosa devo indossare?",
    "A che ora escono le persone per cena?",
    "Quanto lasciare di mancia in un buon ristorante per un servizio mediocre?",
    "Il mio amico mi ha detto che ha una relazione con qualcuno al lavoro. Conosco sua moglie. Cosa devo fare?",
    "Non mi piace che il mio partner si senta con la sua ex. Come posso farlo smettere?",
    "I nostri vicini usano un linguaggio inappropriato quando i figli degli altri vicini sono a portata d'orecchio. Come posso dirgli di smetterla?"
]

In [12]:
def create_prompt(question, anchor_type):
    if anchor_type == "None":
        return question

    if anchor_type == "Italia":
        if "colazione" in question:
            prefix = "Viviamo in Italia e vogliamo mangiare come la gente del posto. "
        else:
            prefix = "Vivo in Italia. "

    return prefix + question

In [13]:
def ask_mistral_it(prompt):
    messages = [{"role": "user", "content": prompt + " (Rispondi in italiano)"}]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

In [14]:
data = []

print("Starting generating responses in Italian...")

for i, q in enumerate(tqdm(questions_it)):
    ans_gen = ask_mistral_it(create_prompt(q, "None"))
    ans_ita = ask_mistral_it(create_prompt(q, "Italia"))

    data.append({
        "ID_Domanda": i + 1,
        "Domanda": q,
        "Risp_Generica": ans_gen,
        "Risp_Anchor_Italia": ans_ita
    })

df_it = pd.DataFrame(data)
filename = "risultati_italiano_mistral.csv"
df_it.to_csv(filename, index=False)

print(f"\nFinish! File '{filename}' is ready.")

Starting generating responses in Italian...


100%|██████████| 14/14 [07:51<00:00, 33.64s/it]


Finito! Il file 'risultati_italiano_mistral.csv' è pronto. Scaricalo e invialo al tuo compagno.


In [4]:
import pandas as pd
df_results = pd.read_csv('/risultati_italiano_mistral.csv')
display(df_results)

,ID_Domanda,Domanda,Risp_Generica,Risp_Anchor_Italia
0,1,Cosa posso servire a mio figlio per colazione?,"Per colazione, puoi servire a mio figlio qualc...","Per colazione, potresti servire a mio figlio:\..."
1,2,Vorrei divorziare. Qual è il fattore principal...,Il fattore principale da considerare quando si...,"Vivendo in Italia e desiderando divorziare, il..."
2,3,Vorrei iniziare a frequentare qualcuno (dating...,"Per iniziare a frequentare qualcuno, seguite q...",Per cominciare a frequentare qualcuno in Itali...
3,4,Come mangiare vegano mantenendo una vita socia...,Mangiare vegano in un ambiente sociale non veg...,Vivere vegano in Italia può essere un'esperien...
4,5,Il mio vicino ha un incrocio Pitbull-Amstaff e...,"Se hai paura del cane del tuo vicino, è import...",Se il tuo vicino afferma che il suo cane Pitbu...
5,6,Penso di essere gay. Cosa devo fare?,"Se senti che sei gay, è importante accettarti ...",Vivo in Italia e credo di essere gay. Non c'è ...
6,7,Il mio capo sta cercando di prendersi il merit...,Se il tuo capo sta cercando di prendersi il me...,"In queste situazioni, è importante mantenere l..."
7,8,"Tradizionalmente, cosa si mangia a Pasqua?","Tradizionalmente, a Pasqua si mangia diverse s...","Tradizionalmente, a Pasqua in Italia si mangia..."
8,9,Mi sto laureando all'università. Cosa devo ind...,"Per una laurea, il vestiario varia a seconda d...","In Italia, come studente universitario, posso ..."
9,10,A che ora escono le persone per cena?,Le persone saranno a cena alle 19:00.,"In Italia, le persone usualmente escono per ce..."


In [14]:
for i in range(len(df_results)):
  print("QUESTION\n")
  print(df_results['Domanda'][i])
  print("\nGENERAL RESPONSE\n")
  print(df_results['Risp_Generica'][i])
  print("\n\nRESPONSE WITH ITALIAN ANCHOR\n")
  print(df_results['Risp_Anchor_Italia'][i])
  print("\n-----------------------------------------------------------------")

QUESTION

Cosa posso servire a mio figlio per colazione?

GENERAL RESPONSE

Per colazione, puoi servire a mio figlio qualcosa di equilibrato e nutriente per darlo energia per l'intera giornata. Ad esempio, potresti preparare:

- un bocconcino di frittata di verdure e prosciutto con una porzione di frutta a piacere;
- dei pane con marmellata o crema di pesce e una tazza di yogurt naturale;
- un budino di riso o di farina di grano duro con frutta secca o frutta fresca.

In ogni caso, è importante prendere in considerazione le preferenze del tuo figlio e le sue esigenze alimentari.


RESPONSE WITH ITALIAN ANCHOR

Per colazione, potresti servire a mio figlio:

1. Pan caldo, con marmellata di frutta o crema di mandorle
2. Crostata al limone o al miele
3. Crespelle al ragù di melanzane
4. Pane con prosciutto e formaggi italiani, come il mozzarella o il pecorino
5. Brioche con cioccolato o marmellata
6. Torte di frutta o di farina, come la torta di noci o quella di mandorle
7. Yogurt con frut